In [167]:
import numpy as np
import regex as re
import pandas as pd

In [2]:
import requests

url_test = 'https://oda.ft.dk/api/Afstemning?$expand=Sagstrin/Sag&$filter=vedtaget eq true&$inlinecount=allpages'

r = requests.get(url_test)

sager = r.json()

In [3]:
#This function takes dict, extracts the next link, gets the new dict and appends it to the old dict

def get_all_pages(url):

    r = requests.get(url)

    sager = r.json()
    prev = r.json()

    while 'odata.nextLink' in prev:

        r = requests.get(prev['odata.nextLink'])
        sager['value'] += r.json()['value']
        prev = r.json()

    return sager

In [19]:
sager = get_all_pages(url_test)
sager.keys()

dict_keys(['odata.metadata', 'odata.count', 'value', 'odata.nextLink'])

In [20]:
#checking if all sager have the same keys
test = [len(sager['value'][i].keys()) for i in range(len(sager['value']))]
np.unique(test)

array([ 9, 10])

In [22]:
#they don't, so we check how many sager have missing keys
missing_keys = [i for i in range(len(sager['value'])) if len(sager['value'][i].keys()) != 10]
missing_keys

[0,
 1099,
 1100,
 1327,
 1404,
 1531,
 1532,
 1533,
 1542,
 1544,
 1550,
 1556,
 1559,
 1560]

In [23]:
#the above is only 14, so we remove those sager
clean_sager = [sag for sag in sager['value'] if len(sag) == 10]

In [81]:
clean_sager[0]['id']

2

In [79]:
clean_sager[0]['Sagstrin']['Sag'].keys()


dict_keys(['id', 'typeid', 'kategoriid', 'statusid', 'titel', 'titelkort', 'offentlighedskode', 'nummer', 'nummerprefix', 'nummernumerisk', 'nummerpostfix', 'resume', 'afstemningskonklusion', 'periodeid', 'afgørelsesresultatkode', 'baggrundsmateriale', 'opdateringsdato', 'statsbudgetsag', 'begrundelse', 'paragrafnummer', 'paragraf', 'afgørelsesdato', 'afgørelse', 'rådsmødedato', 'lovnummer', 'lovnummerdato', 'retsinformationsurl', 'fremsatundersagid', 'deltundersagid'])

In [136]:
#This function takes an id for a vote and collects all the votes for that vote

def get_all_votes(id):
    url = f'https://oda.ft.dk/api/Afstemning({id})/Stemme?$inlinecount=allpages'
    r = requests.get(url)
    votes = r.json()
    prev = r.json()
    while 'odata.nextLink' in prev:
        r = requests.get(prev['odata.nextLink'])
        votes['value'] += r.json()['value']
        prev = r.json()
        
    return votes


votes_test = get_all_votes(clean_sager[0]['id'])

In [137]:
votes_test['value']

[{'id': 180,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 158,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 181,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 126,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 182,
  'typeid': 2,
  'afstemningid': 2,
  'aktørid': 71,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 183,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 50,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 184,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 49,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 185,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 220,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 186,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 183,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 187,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 85,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 188,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 303,
  'opdate

In [177]:
#takes stemme-id which is id in stemme and returns info on the specific politician

def get_politician(id):
    
        url = f'https://oda.ft.dk/api/Stemme({id})/Akt%C3%B8r?$expand=Parti'
        r = requests.get(url)

        raw =  r.json()
        
        politician = dict()
        politician['navn'] = raw['navn']        
        party = re.search("<party>(.*?)<\/party>", raw['biografi']).group(1)
        politician['party'] = party

        return politician



        # we are only interested in typeid: 5, which is a person

test = get_politician(1)

test

{'navn': 'Eigil Andersen', 'party': 'Socialistisk Folkeparti'}

In [115]:
import pandas as pd
import numpy as np


import pandas as pd
def voting_df(data):
    df = pd.DataFrame()

    df['afstemnings_id'] = [x['id'] for x in data]
    df['sagstrin_id'] = [x['sagstrinid'] for x in data]
    df['vedtaget'] = [x['vedtaget'] for x in data]
    df['dato'] = [x['opdateringsdato'] for x in data]
    df['sag_id'] = [x['Sagstrin']['Sag']['id'] for x in data]
    df['titel']= [x['Sagstrin']['Sag']['titel'] for x in data]
    df['titel_kort'] = [x['Sagstrin']['Sag']['titelkort'] for x in data]
    df['lovnummer_num'] = [x['Sagstrin']['Sag']['nummernumerisk'] for x in data]
    df['lovnummer'] = [x['Sagstrin']['Sag']['nummer'] for x in data]
    df['resume']= [x['Sagstrin']['Sag']['resume'] for x in data]
    df['periode_id']= [x['Sagstrin']['Sag']['periodeid'] for x in data]

    return df
    

        
df_voting = voting_df(clean_sager)
df_voting.head()

,afstemnings_id,sagstrin_id,vedtaget,dato,sag_id,titel,titel_kort,lovnummer_num,lovnummer,resume,periode_id
0,2,4849,True,2014-09-09T09:25:05.717,1449,Forslag til lov om ændring af virksomhedsskatt...,Om indgreb mod utilsigtet udnyttelse af virkso...,200,L 200,Loven ændrer virksomhedsskatteordningens regle...,32
1,3,17351,True,2018-01-24T16:46:33.99,5604,"Folketinget konstaterer, at Danmark stod dårli...",Om rigets tilstand.,2,V 2,,31
2,4,18370,True,2018-01-25T10:25:25.64,5904,Forslag til lov om akutjob og jobpræmie til ar...,Om akutjob.,46,L 46,"Med loven etableres en midlertidig ordning, så...",31
3,5,4849,True,2017-08-10T12:57:52.27,1449,Forslag til lov om ændring af virksomhedsskatt...,Om indgreb mod utilsigtet udnyttelse af virkso...,200,L 200,Loven ændrer virksomhedsskatteordningens regle...,32
4,6,16581,True,2017-08-10T12:57:52.257,5384,Forslag til folketingsbeslutning om dansk mili...,Om dansk militært bidrag til støtte for indsat...,122,B 122,Med folketingsbeslutningen giver Folketinget s...,32


In [113]:
df["afstemnings_id"].values

array([   2,    3,    4, ..., 8999, 9000, 9001], dtype=int64)

In [171]:
#This function takes in a df of each vote and makes a dataframe of all the votes for each vote

def votes_df(voting_df):
    df = pd.DataFrame()

    #typeid: 1=ja, 2=nej, 3=blank, 4=fraværende

    df["afstemnings_id"] = voting_df["afstemnings_id"].values
    df["votes_combined_info"] = [get_all_votes(x)['value'] for x in voting_df['afstemnings_id']] #creates a dict
    
    return df



    


In [172]:
votes_df = votes_df(df_voting)

In [174]:
votes_df["votes_combined_info"][0]

[{'id': 180,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 158,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 181,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 126,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 182,
  'typeid': 2,
  'afstemningid': 2,
  'aktørid': 71,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 183,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 50,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 184,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 49,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 185,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 220,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 186,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 183,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 187,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 85,
  'opdateringsdato': '2014-09-09T09:25:05.717'},
 {'id': 188,
  'typeid': 1,
  'afstemningid': 2,
  'aktørid': 303,
  'opdate

In [192]:
votes_df["votes_combined_info"][0][1]

{'id': 181,
 'typeid': 1,
 'afstemningid': 2,
 'aktørid': 126,
 'opdateringsdato': '2014-09-09T09:25:05.717'}

In [189]:
test = [get_politician(person)["id"] for person in votes_df["votes_combined_info"][0]]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)